In [4]:
import torch
import sionna
from sionna import utils
#from sionna.channel import AWGNChannel

from sionna.modulation import QAM
from sionna.ofdm import OFDMModulator, OFDMDemodulator
from sionna.tools import binary_source

# Parameters
num_symbols = 1000      # Number of data symbols
mod_order = 16          # Modulation order (QAM)
snr_db = 20             # Signal to noise ratio in dB
cyclic_prefix_length = 16  # Length of the cyclic prefix
preamble_length = 1        # Length of preamble for timing synchronization

# Generate random binary data source
data = binary_source(num_symbols, mod_order)

# QAM modulation of the data
modulator = QAM(mod_order)
modulated_data = modulator.modulate(data)

# Create OFDM modulator
ofdm_modulator = OFDMModulator(cyclic_prefix_length=cyclic_prefix_length)

# Modulate the data into OFDM symbols
ofdm_symbols = ofdm_modulator(modulated_data)

# Add an AWGN channel
channel = sn.channel.AWGN()
received_signal = channel(ofdm_symbols)

# Timing Synchronization (Correlation with known preamble)
preamble = ofdm_symbols[:preamble_length]  # Simplified assumption: preamble is the first symbol
correlation = torch.correlate(received_signal, preamble, mode='full')
timing_offset = correlation.argmax()

# Synchronized signal after timing correction
synchronized_signal = received_signal[timing_offset:]

# Frequency Synchronization (Using Pilot Tones)
# Assume we add pilot tones (e.g., known QAM symbols) to the signal
pilot_tones = synchronized_signal[::10]  # Pick every 10th symbol as pilot tones

# Frequency offset simulation (e.g., Doppler shift)
frequency_offset = 0.1
received_pilots = pilot_tones * torch.exp(1j * 2 * torch.pi * frequency_offset * torch.arange(len(pilot_tones)))

# Estimate frequency offset
frequency_estimate = torch.angle(torch.mean(received_pilots * torch.conj(pilot_tones)))

# Correct the frequency offset
corrected_signal = synchronized_signal * torch.exp(-1j * frequency_estimate)

# OFDM Demodulation
ofdm_demodulator = OFDMDemodulator(cyclic_prefix_length=cyclic_prefix_length)
demodulated_data = ofdm_demodulator(corrected_signal)

# Display the synchronization results
print(f"Timing Offset: {timing_offset}")
print(f"Estimated Frequency Offset: {frequency_estimate}")


ModuleNotFoundError: No module named 'sionna.modulation'